## Deploying a Model to AWS Sagemaker

In [9]:
import boto3
import mlflow.sagemaker as mfs
import json

app_name = "mlops-sagemaker"
execution_role_arn = "arn:aws:iam::194301488911:role/SageMakerFullAccess"
image_ecr_url = "194301488911.dkr.ecr.eu-west-2.amazonaws.com/mlflow-pyfunc:1.21.0"
region = "eu-west-2"

s3_bucket_name = "mlops-sagemaker-runs-rhys"
experiment_id = "1"
run_id = "10273bfd43124a54a23835dcf1695af4"
model_name = "log_reg_model"

model_uri = "s3://{}/{}/{}/artifacts/{}/".format(s3_bucket_name,experiment_id,run_id,model_name)

In [10]:
mfs.deploy(app_name=app_name,
           model_uri=model_uri,
          execution_role_arn=execution_role_arn,
          region_name=region,
          image_url=image_ecr_url,
          mode=mfs.DEPLOYMENT_MODE_CREATE) # CREATE

2021/11/21 19:02:09 INFO mlflow.sagemaker: Using the python_function flavor for deployment!
2021/11/21 19:02:09 INFO mlflow.sagemaker: No model data bucket specified, using the default bucket
2021/11/21 19:02:10 INFO mlflow.sagemaker: Default bucket `mlflow-sagemaker-eu-west-2-194301488911` already exists. Skipping creation.
2021/11/21 19:02:10 INFO mlflow.sagemaker: tag response: {'ResponseMetadata': {'RequestId': 'Y1SPSVZRATA63PB8', 'HostId': '/7K2zb2Cn/4UyC3sxAzQItEm2j+4qjIwRvduf8X/xd6sTmCXVw1GW5Kxxnd1aS3MmwyHIJ1n/MU=', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': '/7K2zb2Cn/4UyC3sxAzQItEm2j+4qjIwRvduf8X/xd6sTmCXVw1GW5Kxxnd1aS3MmwyHIJ1n/MU=', 'x-amz-request-id': 'Y1SPSVZRATA63PB8', 'date': 'Sun, 21 Nov 2021 19:02:11 GMT', 'server': 'AmazonS3', 'content-length': '0'}, 'RetryAttempts': 0}}
2021/11/21 19:02:10 INFO mlflow.sagemaker: Creating new endpoint with name: mlops-sagemaker ...
2021/11/21 19:02:10 INFO mlflow.sagemaker: Created model with arn: arn:aws:sagemaker:eu-west-2

## Making Predictions

In [20]:
def query(input_json):
    
    client = boto3.session.Session().client("sagemaker-runtime",region)
    
    response = client.invoke_endpoint(
    EndpointName=app_name,
    Body=input_json,
    ContentType='application/json; format=pandas-split')
    
    preds = response['Body'].read().decode("ascii")
    preds = json.loads(preds)
    return preds

In [21]:
import pandas as pd
import mlflow
import mlflow.sklearn

import seaborn as sns

import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix

import numpy as np

df = pd.read_csv("data/creditcard.csv")

In [22]:
normal = df[df.Class == 0].sample(frac=0.5,random_state=2020).reset_index(drop=True)
anomaly = df[df.Class == 1]

normal_train, normal_test = train_test_split(normal, test_size = 0.2, random_state = 2020)
anomaly_train, anomaly_test = train_test_split(anomaly, test_size = 0.2, random_state = 2020)

scaler = StandardScaler()
scaler.fit(pd.concat((normal,anomaly)).drop(["Time","Class"],axis=1))

StandardScaler()

In [23]:
scaled_selection = scaler.transform(df.iloc[:80].drop(["Time","Class"],axis=1))
input_json = pd.DataFrame(scaled_selection).to_json(orient="split")

pd.DataFrame(query(input_json)).T

,0,1,2,3,4,5,6,7,8,9,...,70,71,72,73,74,75,76,77,78,79
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
test = pd.concat((normal.iloc[:1900],anomaly.iloc[:100]))
true = test.Class
test = scaler.transform(test.drop(["Time","Class"],axis=1))
preds = []

batch_size = 80
for f in range(25):
    print(f"Batch {f}", end=" - ")
    
    sample = pd.DataFrame(test[f*batch_size:(f+1)*batch_size]).to_json(orient="split")
    output = query(sample)
    resp = pd.DataFrame([output])
    preds = np.concatenate((preds,resp.values[0]))
    
    print("Completed")
    
eval_acc = accuracy_score(true,preds)
eval_auc = roc_auc_score(true,preds)

print("Eval Acc", eval_acc)
print("Eval AUC", eval_auc)

Batch 0 - Completed
Batch 1 - Completed
Batch 2 - Completed
Batch 3 - Completed
Batch 4 - Completed
Batch 5 - Completed
Batch 6 - Completed
Batch 7 - Completed
Batch 8 - Completed
Batch 9 - Completed
Batch 10 - Completed
Batch 11 - Completed
Batch 12 - Completed
Batch 13 - Completed
Batch 14 - Completed
Batch 15 - Completed
Batch 16 - Completed
Batch 17 - Completed
Batch 18 - Completed
Batch 19 - Completed
Batch 20 - Completed
Batch 21 - Completed
Batch 22 - Completed
Batch 23 - Completed
Batch 24 - Completed
Eval Acc 0.9915
Eval AUC 0.915


## Switching Models

In [27]:
new_run_id = "52906ffde7a940dcb285fbb8aad88643"

new_model_uri = "s3://{}/{}/{}/artifacts/{}/".format(s3_bucket_name,experiment_id,new_run_id,model_name)

mfs.deploy(app_name=app_name,
          model_uri=new_model_uri,
          execution_role_arn=execution_role_arn,
          region_name=region,
          image_url=image_ecr_url,
          mode=mfs.DEPLOYMENT_MODE_REPLACE) # REPLACE

2021/11/21 19:53:19 INFO mlflow.sagemaker: Using the python_function flavor for deployment!
2021/11/21 19:53:19 INFO mlflow.sagemaker: No model data bucket specified, using the default bucket
2021/11/21 19:53:20 INFO mlflow.sagemaker: Default bucket `mlflow-sagemaker-eu-west-2-194301488911` already exists. Skipping creation.
2021/11/21 19:53:21 INFO mlflow.sagemaker: tag response: {'ResponseMetadata': {'RequestId': '644ZRE84H6DK59VC', 'HostId': 'Pq7b/dJ7attFh+foMd9KTB2kQxDCHV56CguMtsINMtgem8ZAv+/4N8C4rNZi22jqE7h+R6U9yrE=', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': 'Pq7b/dJ7attFh+foMd9KTB2kQxDCHV56CguMtsINMtgem8ZAv+/4N8C4rNZi22jqE7h+R6U9yrE=', 'x-amz-request-id': '644ZRE84H6DK59VC', 'date': 'Sun, 21 Nov 2021 19:53:22 GMT', 'server': 'AmazonS3', 'content-length': '0'}, 'RetryAttempts': 0}}
2021/11/21 19:53:21 INFO mlflow.sagemaker: Found active endpoint with arn: arn:aws:sagemaker:eu-west-2:194301488911:endpoint/mlops-sagemaker. Updating...
2021/11/21 19:53:21 INFO mlflow.sage

In [29]:
test = pd.concat((normal.iloc[:1900],anomaly.iloc[:100]))
true = test.Class
test = scaler.transform(test.drop(["Time","Class"],axis=1))
preds = []

batch_size = 80
for f in range(25):
    print(f"Batch {f}", end=" - ")
    
    sample = pd.DataFrame(test[f*batch_size:(f+1)*batch_size]).to_json(orient="split")
    output = query(sample)
    resp = pd.DataFrame([output])
    preds = np.concatenate((preds,resp.values[0]))
    
    print("Completed")
    
eval_acc = accuracy_score(true,preds)
eval_auc = roc_auc_score(true,preds)

print("Eval Acc", eval_acc)
print("Eval AUC", eval_auc)

Batch 0 - Completed
Batch 1 - Completed
Batch 2 - Completed
Batch 3 - Completed
Batch 4 - Completed
Batch 5 - Completed
Batch 6 - Completed
Batch 7 - Completed
Batch 8 - Completed
Batch 9 - Completed
Batch 10 - Completed
Batch 11 - Completed
Batch 12 - Completed
Batch 13 - Completed
Batch 14 - Completed
Batch 15 - Completed
Batch 16 - Completed
Batch 17 - Completed
Batch 18 - Completed
Batch 19 - Completed
Batch 20 - Completed
Batch 21 - Completed
Batch 22 - Completed
Batch 23 - Completed
Batch 24 - Completed
Eval Acc 0.9915
Eval AUC 0.915


## Removing Deployed Model

In [30]:
mfs.delete(app_name=app_name,region_name=region)

2021/11/21 20:04:00 INFO mlflow.sagemaker: Deleted endpoint with arn: arn:aws:sagemaker:eu-west-2:194301488911:endpoint/mlops-sagemaker
2021/11/21 20:04:00 INFO mlflow.sagemaker: Waiting for the delete operation to complete...
2021/11/21 20:04:00 INFO mlflow.sagemaker: Deletion is still in progress. Current endpoint status: Deleting
2021/11/21 20:04:05 INFO mlflow.sagemaker: The deletion operation completed successfully with message: "The SageMaker endpoint was deleted successfully."
2021/11/21 20:04:05 INFO mlflow.sagemaker: Cleaning up unused resources...
2021/11/21 20:04:05 INFO mlflow.sagemaker: Deleted associated endpoint configuration with arn: arn:aws:sagemaker:eu-west-2:194301488911:endpoint-config/mlops-sagemaker-config-okmwtxbvstwrmjwt71y11w
2021/11/21 20:04:06 INFO mlflow.sagemaker: Deleted associated model with arn: arn:aws:sagemaker:eu-west-2:194301488911:model/mlops-sagemaker-model-wd5jt8nestioyaeabcwkqna
